<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


*   Inputs: A question q = {q1, ..., qQ} of length Q and a context paragraph p = {p1, ..., pP } of length P.
*   Output: An answer span {as, ae} where as is the index of the first answer token in p, ae is
the index of the last answer token in p, 0 <= as, ae >= m, and as >= ae.



# Import Libraries and Read Data

In [4]:
import tensorflow as tf
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.head(2)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer,answer_len,answer_end,answer_span
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,when did beyonce start becoming popular,in the late 1990s,17,286,"(269, 286)"
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,what areas did beyonce compete in when she was...,singing and dancing,19,226,"(207, 226)"


In [68]:
squad_df[squad_df["answer_span"] == (-1,-1)]

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer,answer_len,answer_end,answer_span


# Lets use 2 LSTM network for context and question and use a concat layer to merge. y is the answer 

# Step 1 Tokenize and Vectorize

### Tokenizing context and question

In [5]:
from tqdm import tqdm
tokenizer = preprocessing.text.Tokenizer(num_words=80000)

for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):  
  tokenizer.fit_on_texts(text.values)

100%|██████████| 2/2 [00:09<00:00,  4.64s/it]


In [6]:
print(tokenizer.num_words)

80000


In [0]:
# tokenizer = text.Tokenizer()

# for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):
#     tokenizer.fit_on_texts(text.values)

# context_tokenize = preprocessing.text.Tokenizer()
# context_tokenize.fit_on_texts(squad_df['clean_context']) #Fit it on clean_context

# questions_tokenize= preprocessing.text.Tokenizer()
# questions_tokenize.fit_on_texts(squad_df['clean_question'])

In [0]:
### SAVE TOKENIZERS
pickle.dump(tokenizer, open(project_path + "tokenizer.pickle", "wb"))
# pickle.dump(questions_tokenize, open(project_path + "tokenizer.pickle", "wb"))

### Vectorization 

In [0]:
#convert the context text to indexes
context_sequence= tokenizer.texts_to_sequences(squad_df['clean_context'])
# convert the questions to indexes
questions_sequence= tokenizer.texts_to_sequences(squad_df['clean_question'])

### Find Max Sequence Length for both the Sequences

In [13]:
# max length of context
max_context_seq_length= max(len(txt) for txt in context_sequence)
print('max_context_seq_length=',max_context_seq_length)

# vocab size of context
vocab_size=len(tokenizer.word_index)
print('vocab_size=',vocab_size)

max_question_seq_length=max(len(txt) for txt in questions_sequence)
print('max_question_seq_length=',max_question_seq_length)


max_context_seq_length= 426
vocab_size= 100850
max_question_seq_length= 40


In [0]:
# From the EDA and historgrams we can conclude that - 
# 99% percentile of context word length = 285
# 99% percentile or question word lengt = 20
max_context_seq_length = 285
max_question_seq_length = 20

In [15]:
print(squad_df['clean_context'][2000])
print(context_sequence[3000])
print(squad_df['clean_question'][2000])
print(questions_sequence[2000])

october 21 2008 apple reported 14 21 total revenue fiscal quarter 4 year 2008 came ipods september 9 2009 keynote presentation apple event phil schiller announced total cumulative sales ipods exceeded 220 million continual decline ipod sales since 2009 surprising trend apple corporation apple cfo peter oppenheimer explained june 2009 expect traditional mp3 players decline time cannibalize ipod touch iphone since 2009 companys ipod sales continually decreased every financial quarter 2013 new model introduced onto market
[384, 511, 58838, 520, 504, 34, 7655, 7507, 1434, 2740, 3028, 8, 22, 1434, 5566, 48288, 1070, 2828, 4793, 266, 1211, 9403, 2828, 5566, 7837, 3016, 1636, 4027, 697, 1922, 5566, 252, 2271, 4793, 27639, 99, 578, 5016, 142, 1434, 680, 5683, 142, 5566, 7, 58839, 1562, 11769, 4284, 17681, 255, 58840, 1050, 3760, 5209, 24179, 1084, 2740, 3028, 235, 1376, 115, 6740, 179, 1578, 374, 4793, 27639, 578, 8588, 44, 3439, 19, 455, 2820, 574, 193, 578, 87, 13, 11280, 60, 3366, 636]
who 

### Padding the sequences

In [16]:
# padding context
context_input_data= tf.keras.preprocessing.sequence.pad_sequences(context_sequence, maxlen=max_context_seq_length, padding='pre')
# padding question
question_input_data=tf.keras.preprocessing.sequence.pad_sequences(questions_sequence, maxlen=max_question_seq_length,padding='pre')

print(context_input_data.shape)
print(question_input_data.shape)

(130306, 285)
(130306, 20)


# Build the LSTM Model for both Sequence

In [0]:
embedding_size = 50
rnn_units=256

### Embedding Layer for Context and Question

In [0]:
# CONTEXT LSTM
# input layer
context_input=layers.Input(shape=(max_context_seq_length,),name="CONTEXT_INPUT")
# Build Embedding layer and Get Embedding Layer output

context_embedding_output=layers.Embedding(input_dim=tokenizer.num_words+1, 
                                          output_dim=embedding_size, 
                                          input_length=max_context_seq_length,
                                          name="CONTEXT_EMBEDDING")(context_input)


# QUESTION LSTM
#input layer
question_input=layers.Input(shape=(max_question_seq_length,),name="QUESTION_INPUT")
#Embedding layer and #Embedding layer output
question_embedding_output=layers.Embedding(input_dim=tokenizer.num_words+1, 
                                           output_dim=embedding_size, 
                                           input_length=max_question_seq_length,
                                           name="QUESTION_EMBEDDING")(question_input)

### Encoder Layer for Context and Question

In [0]:
# RNN Encoder with LSTM for context
c_output,c_h, c_s = layers.LSTM(rnn_units,name='CONTEXT_LSTM', return_state=True)(context_embedding_output)
context_states= [c_h, c_s]

# RNN Encoder with LSTM for question
q_output,q_h, q_s= tf.keras.layers.LSTM(rnn_units,name='QUESTION_LSTM',return_state=True)(question_embedding_output)
questions_states = [q_h, q_s]

In [57]:
context_states,questions_states

([<tf.Tensor 'CONTEXT_LSTM/Identity_1:0' shape=(None, 256) dtype=float32>,
  <tf.Tensor 'CONTEXT_LSTM/Identity_2:0' shape=(None, 256) dtype=float32>],
 [<tf.Tensor 'QUESTION_LSTM/Identity_1:0' shape=(None, 256) dtype=float32>,
  <tf.Tensor 'QUESTION_LSTM/Identity_2:0' shape=(None, 256) dtype=float32>])

### Concat the both RNN LSTM Encoder layers to get merged cell state and hidden state

In [0]:
MERGED_cell_state =layers.concatenate([context_states[0],questions_states[0]],name="CONCAT_CELL_STATE")
MERGED_hidden_state =layers.concatenate([context_states[1],questions_states[1]],name="HIDDEN_CELL_STATE")

In [14]:
decoder_initial_state = [MERGED_cell_state,MERGED_hidden_state]
decoder_initial_state

[<tf.Tensor 'CONCAT_CELL_STATE/Identity:0' shape=(None, 512) dtype=float32>,
 <tf.Tensor 'HIDDEN_CELL_STATE/Identity:0' shape=(None, 512) dtype=float32>]

# Create Decoder for Answer

# Add  Start and  End tokens to Answers

In [0]:
squad_df['answer_start_end']= '<start>' + squad_df['clean_answer'] + '<end>'
squad_df['answer_start_end']=squad_df['answer_start_end'].astype(str)

#Tokenize the Answers

In [0]:
answers_tokenize=tf.keras.preprocessing.text.Tokenizer()
answers_tokenize.fit_on_texts(squad_df['answer_start_end'])

In [17]:
#Vocab
print(len(answers_tokenize.word_index))

41475


In [0]:
#Convert sentences to numbers 
answers_seq = answers_tokenize.texts_to_sequences(squad_df['answer_start_end']) 

In [19]:
print(squad_df['answer_start_end'][2000])
print(answers_seq[2000])

<start>peter oppenheimer<end>
[2, 924, 7781, 1]


# Get maximum length and pad the sequences

In [20]:
squad_df[squad_df['clean_answer'].str.len() > 200]['answer_start_end']

3201    <start>that the sudden shift of a huge quantit...
Name: answer_start_end, dtype: object

In [21]:
max_answers_seq_length=max(len(txt) for txt in squad_df['answer_start_end'])
print('max_answers_seq_length=',max_answers_seq_length)

answers_vocab_size=len(answers_tokenize.word_index)
print('answers_vocab_size=',answers_vocab_size)

max_answers_seq_length= 248
answers_vocab_size= 41475


In [0]:
# From the EDA and historgrams we can conclude that - 
# 99% percentile of answer word length = 17
max_answers_seq_length=17

In [0]:
# pad pre
answers_input_data= tf.keras.preprocessing.sequence.pad_sequences(answers_seq,maxlen=max_answers_seq_length,padding='pre')

# Building Decoder Output

In [24]:
answers_input_data.shape

(130306, 17)

In [0]:
#Initialize array
answers_target_data = np.zeros((answers_input_data.shape[0], #number of sentences 130306
                                answers_input_data.shape[1])) #number of words in each sentence 248

#Shift Target output by one word
for i in range(answers_input_data.shape[0]):
    for j in range(1,answers_input_data.shape[1]):
        answers_target_data[i][j-1] = answers_input_data[i][j]

In [26]:
print(squad_df['answer_start_end'][2000])
print(answers_input_data[2000])

<start>peter oppenheimer<end>
[   0    0    0    0    0    0    0    0    0    0    0    0    0    2
  924 7781    1]


# Convert Answers to one-hot vector

In [0]:
## Crashing !!
answers_target_data_one_hot= np.zeros((answers_input_data.shape[0], #number of sentences
                                       answers_input_data.shape[1], #Number of words in each sentence
                                       len(answers_tokenize.word_index)+1)) #Vocab size + 1

In [27]:
print(answers_input_data.shape)
print(len(answers_tokenize.word_index)+1)

(130306, 17)
41476


In [0]:
answers_embedding_size = 50
decoder_rnn_units = 512

# Build Decoder

In [0]:
#input layer
answers_inputs=tf.keras.layers.Input(shape=(max_answers_seq_length,),name="ANSWER_INPUT")

#Embedding
answers_embedding_output=tf.keras.layers.Embedding(answers_vocab_size+1, answers_embedding_size, name="ANSWER_EMBEDDING")(answers_inputs)

#lstm layer
answers_lstm= tf.keras.layers.LSTM(decoder_rnn_units,return_sequences=True,name="ANSWER_LSTM", return_state=True)

#LSTM Output, State initialization from Encoder states(concat of question and answer)
#Output will be all hidden sequences, last 'h' state and last 'c' state

output,_,_=answers_lstm(answers_embedding_output,initial_state=decoder_initial_state)

#dense layer
lstm3_dense= tf.keras.layers.Dense(answers_vocab_size+1,activation='softmax',name="FINAL_OUTPUT")

#answer output
answer_outputs=lstm3_dense(output)

# Build Model using Encoder ( output of concat) and Decoder

In [0]:
model = tf.keras.models.Model([context_input,question_input, answers_inputs],answer_outputs) #Output of the model

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
CONTEXT_INPUT (InputLayer)      [(None, 285)]        0                                            
__________________________________________________________________________________________________
QUESTION_INPUT (InputLayer)     [(None, 20)]         0                                            
__________________________________________________________________________________________________
CONTEXT_EMBEDDING (Embedding)   (None, 285, 50)      4676500     CONTEXT_INPUT[0][0]              
__________________________________________________________________________________________________
QUESTION_EMBEDDING (Embedding)  (None, 20, 50)       2364500     QUESTION_INPUT[0][0]             
______________________________________________________________________________________________

# Train the Model